**clean up of the individual and the combined files**

In [1]:
#load all the relevant packages
import pandas as pd
import numpy as np
import os
import re
import datetime
import seaborn as sns
import matplotlib as mlt
import json

In [2]:
#set directory
print(os.getcwd())
os.chdir('C:\\Users\\cecilianatalie.strom\\GitHub\\INSA\\Opensanctions\\')
print(os.getcwd())

C:\Users\cecilianatalie.strom\GitHub\INSA\Opensanctions\Clean
C:\Users\cecilianatalie.strom\GitHub\INSA\Opensanctions


# load the dataframe

In [3]:
df = pd.read_csv('old files\\df_complete.csv',low_memory=False)

# clean up country and iso columns

In [4]:
df.countries.unique()

array(['sy', 'de', 'ru', nan, 'zw', 'pk', 'ir', 'mx', 'co', 'cn', 'tw',
       'kp', 've', 'ps', 'gb', 'mt', 'vc', 'ni', 'mx;us', 'jp', 'tn',
       'cy;gb;gh;gr;sy', 'km', 'eg', 'sk;ve', 'lb', 'au', 'af;ir;pk',
       'ae', 'mm', 'iq', 'cy', 'cr', 'jo', 'ar', 'ca', 'rs', 'it;tn',
       'sv', 'at', 'lv;ru', 'tr', 'gb;sd', 'cn;kp', 'pa', 'af;pk', 'ph',
       'ye', 'ae;lr', 'de;jo', 'de;iq', 'cn;hk', 'es', 'gt;pa;sv', 'vg',
       'do', 'ly', 'cz;sc', 'kh', 'th', 'gt', 'us', 'gr', 'gb;jo',
       'ao;ar;bb;ca;cz;de;es;fr;gb;gy;ht;jm;mx;pa;pe;ru', 'ua', 'iq;it',
       'br;cn;eg;kp;pe;ru;th', 'mh;tr', 'tg', 'tz', 'me;rs', 'ch',
       'co;mx', 'bg', 'co;lb', 'ws', 'ae;so', 'om;so', 'lr', 'ch;ir',
       'hn', 'gb;jo;us', 'sa', 'se', 'ir;sy', 'sd', 'ae;gb', 'ae;sc',
       'af', 'fr;lb', 'sg', 'ss', 'ky', 'ec', 'gb;hk;ir', 'id', 'mn',
       'co;pa', 'cn;hk;mm', 'li', 'im', 'cn;tw', 'co;cr', 'hk',
       'co;lb;ve', 'lb;us', 'ir;my', 'ke', 'es;ve', 'bj', 'ru;ua',
       'cy;vg', 'ae;in',

In [5]:
iso =pd.read_csv('Clean\\country-codes_csv.csv')
iso = iso.filter(['ISO3166-1-Alpha-3', 'MARC'])
iso=iso.rename(columns={'MARC':'countries','ISO3166-1-Alpha-3':'iso'})
display(iso)

,iso,countries
0,TWN,ch
1,AFG,af
2,ALB,aa
3,DZA,ae
4,ASM,as
...,...,...
245,ESH,ss
246,YEM,ye
247,ZMB,za
248,ZWE,rh


In [6]:
iso.countries.unique()

array(['ch', 'af', 'aa', 'ae', 'as', 'an', 'ao', 'am', 'ay', 'aq', 'ag',
       'ai', 'aw', 'at', 'au', 'aj', 'bf', 'ba', 'bg', 'bb', 'bw', 'be',
       'bh', 'dm', 'bm', 'bt', 'bo', 'ca', 'bn', 'bs', 'bv', 'bl', 'bi',
       'vb', 'bx', 'bu', 'uv', 'bd', 'cv', 'cb', 'cm', 'xxc', 'cj', 'cx',
       'cd', 'cl', 'cc', nan, 'xa', 'xb', 'ck', 'cq', 'cf', 'cw', 'cr',
       'ci', 'cu', 'co', 'cy', 'xr', 'iv', 'kn', 'cg', 'dk', 'ft', 'dq',
       'dr', 'ec', 'ua', 'es', 'eg', 'ea', 'er', 'sq', 'et', 'fk', 'fa',
       'fj', 'fi', 'fr', 'fg', 'fp', 'fs', 'go', 'gm', 'gs', 'gw', 'gh',
       'gi', 'gr', 'gl', 'gd', 'gp', 'gu', 'gt', 'uik', 'gv', 'pg', 'gy',
       'ht', 'hm', 'vc', 'ho', 'hu', 'ic', 'ii', 'io', 'ir', 'iq', 'ie',
       'is', 'it', 'jm', 'ja', 'jo', 'kz', 'ke', 'gb', 'ku', 'kg', 'ls',
       'lv', 'le', 'lo', 'lb', 'ly', 'lh', 'li', 'lu', 'mg', 'mw', 'my',
       'xc', 'ml', 'mm', 'xe', 'mq', 'mu', 'mf', 'ot', 'mx', 'fm', 'mc',
       'mp', 'mo', 'mj', 'mr', 'mz', 'br', 'sx', '

In [7]:
df.countries.unique()

array(['us', 'gb', 'ca', nan, 'ae', 'gb;lb;sy;tr', 'fr', 'de', 'se', 'ru',
       'de;es', 'ly', 'mx', 'cn', 'ir', 'lb', 'in', 'tw', 'sg', 've',
       'ch;fr;ly', 'be', 'tr', 'mv', 'sy', 'ir;us', 'iq;sa', 'in;us',
       'ch', 'gr', 'lu;ly;mt', 'at', 'ru;tr', 'ae;iq;jo', 'bg', 'iq;jo',
       'li', 'by', 'lu', 'ua', 'sg;us', 'iq', 'gb;lb;sy', 'lb;sy;tr'],
      dtype=object)

In [8]:
matches_df.countries.unique()

NameError: name 'matches_df' is not defined

In [ ]:
# Check if the values in 'col1' and 'col2' match
match = df['countries'].equals(iso['countries'])
# Print the result
print(match)

In [ ]:
#check if there are any matches
matches_df = df.copy()
matches_df['test'] = df[df['countries'].isin(iso['countries'])]
print(matches_df)

In [5]:
df.countries.unique()

array(['sy', 'de', 'ru', nan, 'zw', 'pk', 'ir', 'mx', 'co', 'cn', 'tw',
       'kp', 've', 'ps', 'gb', 'mt', 'vc', 'ni', 'mx;us', 'jp', 'tn',
       'cy;gb;gh;gr;sy', 'km', 'eg', 'sk;ve', 'lb', 'au', 'af;ir;pk',
       'ae', 'mm', 'iq', 'cy', 'cr', 'jo', 'ar', 'ca', 'rs', 'it;tn',
       'sv', 'at', 'lv;ru', 'tr', 'gb;sd', 'cn;kp', 'pa', 'af;pk', 'ph',
       'ye', 'ae;lr', 'de;jo', 'de;iq', 'cn;hk', 'es', 'gt;pa;sv', 'vg',
       'do', 'ly', 'cz;sc', 'kh', 'th', 'gt', 'us', 'gr', 'gb;jo',
       'ao;ar;bb;ca;cz;de;es;fr;gb;gy;ht;jm;mx;pa;pe;ru', 'ua', 'iq;it',
       'br;cn;eg;kp;pe;ru;th', 'mh;tr', 'tg', 'tz', 'me;rs', 'ch',
       'co;mx', 'bg', 'co;lb', 'ws', 'ae;so', 'om;so', 'lr', 'ch;ir',
       'hn', 'gb;jo;us', 'sa', 'se', 'ir;sy', 'sd', 'ae;gb', 'ae;sc',
       'af', 'fr;lb', 'sg', 'ss', 'ky', 'ec', 'gb;hk;ir', 'id', 'mn',
       'co;pa', 'cn;hk;mm', 'li', 'im', 'cn;tw', 'co;cr', 'hk',
       'co;lb;ve', 'lb;us', 'ir;my', 'ke', 'es;ve', 'bj', 'ru;ua',
       'cy;vg', 'ae;in',

In [ ]:
df[['marc_codes1','marc_codes2','marc_codes3','marc_codes4']] = df['countries'].str.split(';',expand=True)
df

In [ ]:
# Split the 'marc_codes' column by semicolon into a list of codes

matches_df = pd.merge(df,iso,how='inner')
matches_df

# Use the str.split with expand to create separate columns
df_expanded = matches_df['marc_codes'].apply(lambda x: pd.Series(x)).add_prefix('country_')

# Limit the number of expanded columns to 16
df_expanded = df_expanded.iloc[:, :17]

# Concatenate the new columns with the original DataFrame
df_result = pd.concat([matches_df, df_expanded], axis=1)

# Drop the original 'marc_codes' column if needed
df_result.drop(columns=['marc_codes'], inplace=True)

# Print or use the modified DataFrame as needed
df_result

In [ ]:
#check for completeness
df_result.iso.unique()

# clean up the sanctions column

In [6]:
#get all information in a list
list1=df.sanctions.values.tolist()
list2=pd.unique(list1).tolist()
list2

['Block - Program - SDN List - Executive Order 13572 (Syria) - 2011-05-18',
 'Block - Program - SDN List - Unknown - 2011-04-28',
 'Block - Program - SDN List - Executive Order 13810 (North Korea) - 2017-09-26',
 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - Executive Order 13224 (Terrorism) - 2012-09-19',
 'Block - Program - SDN List - Unknown - 2005-08-03',
 'Block - Program - SDN List - Executive Order 13224 (Terrorism) - 2013-08-20',
 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - CISADA - 2014-08-29',
 'Block - Program - SDN List - Foreign Narcotics Kingpin Designation Act - 2018-03-06',
 'Block - Program - SDN List - Unknown - 1997-01-15',
 'Block - Program - SDN List - Executive Order 13660 (Ukraine) - 2015-03-11',
 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - TRA - 2014-04-29',
 'Block - Program - SDN List - Executive Order 13599 (Iran) - 2013-06-04',
 'Block - Program - SDN List - Executive Ord

In [7]:
print(len(list1))
print(len(list2))

5407
1259


In [8]:
#chunk the list to access all the information on the regimes
n=10
list(zip(*[iter(list2)]*n))
#save as json
with open("Clean\\all_regime.json", 'w') as f:
    json.dump(list2, f) 

with open("Clean\\all_regime.json", 'r') as f:
    bis_sanction = json.load(f)

print(bis_sanction)
#deduplicate sanctions

['Block - Program - SDN List - Executive Order 13572 (Syria) - 2011-05-18', 'Block - Program - SDN List - Unknown - 2011-04-28', 'Block - Program - SDN List - Executive Order 13810 (North Korea) - 2017-09-26', 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - Executive Order 13224 (Terrorism) - 2012-09-19', 'Block - Program - SDN List - Unknown - 2005-08-03', 'Block - Program - SDN List - Executive Order 13224 (Terrorism) - 2013-08-20', 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - CISADA - 2014-08-29', 'Block - Program - SDN List - Foreign Narcotics Kingpin Designation Act - 2018-03-06', 'Block - Program - SDN List - Unknown - 1997-01-15', 'Block - Program - SDN List - Executive Order 13660 (Ukraine) - 2015-03-11', 'Block - Program - SDN List - Executive Order 13382 (Non-proliferation) - TRA - 2014-04-29', 'Block - Program - SDN List - Executive Order 13599 (Iran) - 2013-06-04', 'Block - Program - SDN List - Executive Order 13810 (No

# create year and month ids

In [9]:
# Add a 'year' column
df_result=df.copy()
df_result['year_id'] = pd.to_datetime(df_result['date_stamp'],format='%Y%m%d').dt.to_period('Y')

In [10]:
#transfrom listing_date to datetime
df_result['listing_date']=pd.to_datetime(df_result['listing_date'],format='%Y%m%d')

In [11]:
# Add a 'month' column
df_result=df.copy()
df_result['month_num'] = pd.to_datetime(df_result['listing_date'],format='%Y%m%d').dt.to_period('M')

In [12]:
df_result

,Unnamed: 0.1,Unnamed: 0,id,schema,name,aliases,birth_date,countries,addresses,identifiers,...,new_entry,deletion,unchanged,month,listing_count,listing_date,delisting_date,new_listing_count,last_change,month_num
0,0,0,NK-226JTDehVMSXex35EbKPmi,Organization,BENA PROPERTIES,BANNA PROPERTIES;BENA,NaN,sy,"Cham Holding Building, Daraa Highway, Sahnaya ...",NaN,...,True,False,False,2021-09,147560,20210930,NaN,1,NaN,2021-09
1,1,9,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,...,True,False,False,2021-09,147560,20210930,NaN,1,NaN,2021-09
2,2,21,NK-26KatzEr7TvuaQuVTYUvrJ,Person,Sang Jun JI,Sang-chun Chi,1971-05-03,ru,Moscow,NaN,...,True,False,False,2021-09,147560,20210930,NaN,1,NaN,2021-09
3,3,29,NK-27foGhde2c676yHk4Szsir,Airplane,EX-301,NaN,NaN,NaN,NaN,EX-301,...,True,False,False,2021-09,147560,20210930,NaN,1,NaN,2021-09
4,4,35,NK-28gREYAn2aeBFeu546pXgd,Organization,OLDHAM FARM,NaN,NaN,zw,Chegutu,NaN,...,True,False,False,2021-09,147560,20210930,NaN,1,NaN,2021-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5402,5402,402961,us-bis-12-20-2022-jesse-cortez-arguelles,LegalEntity,JESSE CORTEZ-ARGUELLES,NaN,NaN,NaN,"150 W. LINCOLN STREEET, APT. A, 85714 TUCSON",NaN,...,True,True,False,2022-12,303190,20221220,20221220.0,1,NaN,2022-12
5403,5403,535301,us-bis-12-20-2022-nordwind-airlines,LegalEntity,NORDWIND AIRLINES,NaN,NaN,ru,"LENINGRADSKAYA STR., BUILDING 25, OFFICE 27. 2...",NaN,...,True,False,False,2023-01,306805,20230124,NaN,1,NaN,2023-01
5404,5404,402962,us-bis-12-20-2022-pobeda-airlines,LegalEntity,POBEDA AIRLINES,NaN,NaN,ru,"108811, RUSSIAN FEDERATION, MOSCOW, P. MOSKOVS...",NaN,...,True,True,False,2022-12,303190,20221220,20221220.0,1,NaN,2022-12
5405,5405,402963,us-bis-12-20-2022-priscilla-bustos-martinez,LegalEntity,PRISCILLA BUSTOS MARTINEZ,NaN,NaN,NaN,"7731 GLASGOW DRIVE, 78223 SAN ANTONIO",NaN,...,True,True,False,2022-12,303190,20221220,20221220.0,1,NaN,2022-12


In [13]:
df_result.month.unique()

array(['2021-09', '2023-02', '2023-05', '2022-01', '2021-11', '2023-01',
       '2023-09', '2023-04', '2022-02', '2022-07', '2022-05', '2023-03',
       '2022-06', '2023-07', '2021-12', '2022-03', '2023-06', '2022-04',
       '2022-10', '2022-11', '2022-12', '2023-10', '2023-12', '2023-08',
       '2022-08', '2022-09', '2023-11'], dtype=object)

# save to csv

In [ ]:
#save to csv
df_result.to_csv('Clean\\us_bis.csv')

# now get the other lists

In [ ]:
print('load the dataframe')
df_result = pd.read_csv('old files\\us_ofac_cons.csv',low_memory=False)
print('clean up the sanctions column')
#get all information in a list
list1=df_result.sanctions.values.tolist()
list2=pd.unique(list1).tolist()
n=10
list(zip(*[iter(list2)]*n))
#save as json
with open("Clean\\us_cons_regime.json", 'w') as f:
    json.dump(list2, f) 
with open("Clean\\us_cons_regime.json", 'r') as f:
    cons_sanction = json.load(f)
print(cons_sanction)
# Add a 'year' column
df_result['year_id'] = pd.to_datetime(df_result['date_stamp'],format='%Y%m%d').dt.to_period('Y')
print(df_result.month.unique())
# Convert the 'month' column to periods and factorize to get numerical month IDs
df_result['month_id'] = pd.to_datetime(df_result['month'], format='%Y-%m').dt.to_period('M').factorize()[0] + 1
print(df_result)
#save to csv
df_result.to_csv('Clean\\ofac_cons.csv')

In [ ]:
print('load the dataframe')
df_result = pd.read_csv('old files\\us_ofac_sdn.csv',low_memory=False)

print('clean up the sanctions column')
#get all information in a list
list1=df_result.sanctions.values.tolist()
list2=pd.unique(list1).tolist()
n=10
list(zip(*[iter(list2)]*n))
#save as json
with open("Clean\\us_sdn_regime.json", 'w') as f:
    json.dump(list2, f) 
with open("Clean\\us_sdn_regime.json", 'r') as f:
    sdn_sanction = json.load(f)
print(sdn_sanction)
# Add a 'year' column
df_result['year_id'] = pd.to_datetime(df_result['date_stamp'],format='%Y%m%d').dt.to_period('Y')
print(df_result.month.unique())
# Convert the 'month' column to periods and factorize to get numerical month IDs
df_result['month_id'] = pd.to_datetime(df_result['month'], format='%Y-%m').dt.to_period('M').factorize()[0] + 1
print(df_result)
#save to csv
df_result.to_csv('Clean\\ofac_sdn.csv')

In [ ]:
print('load the dataframe')
df_result = pd.read_csv('old files\\eu_fsf.csv',low_memory=False)

print('clean up the sanctions column')
#get all information in a list
list1=df_result.sanctions.values.tolist()
list2=pd.unique(list1).tolist()
n=10
list(zip(*[iter(list2)]*n))
#save as json
with open("Clean\\eu_fsf_regime.json", 'w') as f:
    json.dump(list2, f) 
with open("Clean\\eu-fsf_regime.json", 'r') as f:
    eu_fsf_sanction = json.load(f)
print(eu_fsf)
# Add a 'year' column
df_result['year_id'] = pd.to_datetime(df_result['date_stamp'],format='%Y%m%d').dt.to_period('Y')
print(df_result.month.unique())
# Convert the 'month' column to periods and factorize to get numerical month IDs
df_result['month_id'] = pd.to_datetime(df_result['month'], format='%Y-%m').dt.to_period('M').factorize()[0] + 1
print(df_result)
#save to csv
df_result.to_csv('Clean\\eu_fsf.csv')

In [ ]:
print('load the dataframe')
df_result = pd.read_csv('old files\\eu_sanctions_map.csv',low_memory=False)
print('clean up the sanctions column')
#get all information in a list
list1=df_result.sanctions.values.tolist()
list2=pd.unique(list1).tolist()
n=10
list(zip(*[iter(list2)]*n))
#save as json
with open("Clean\\eu_map_regime.json", 'w') as f:
    json.dump(list2, f) 
with open("Clean\\eu_map_regime.json", 'r') as f:
    eu_map_sanction = json.load(f)
print(eu_map_sanction)
# Add a 'year' column
df_result['year_id'] = pd.to_datetime(df_result['date_stamp'],format='%Y%m%d').dt.to_period('Y')
print(df_result.month.unique())
# Convert the 'month' column to periods and factorize to get numerical month IDs
df_result['month_id'] = pd.to_datetime(df_result['month'], format='%Y-%m').dt.to_period('M').factorize()[0] + 1
print(df_result)
#save to csv
df_result.to_csv('Clean\\eu_sanctions_map.csv')